<a href="https://colab.research.google.com/github/auzaluis/upsa_mod_202501/blob/main/personalidad/01_script_ETL_personalidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 01: Carga de datos

## Importando base de datos

In [ ]:
# Google Auth
from google.colab import auth
auth.authenticate_user()

In [ ]:
# API client
from google.auth import default
creds, _ = default()

In [ ]:
# gspread authorization
import gspread
gc = gspread.authorize(creds)

In [ ]:
# Accediendo al Google Sheet
url_personalidad = 'https://docs.google.com/spreadsheets/d/1IQ_RxxTSmBKHTExlxboIRNlMov_F6RyqdcOPrflCv_w/edit?usp=sharing'
gsheets = gc.open_by_url(url_personalidad)
sheets = gsheets.worksheet('Respuestas de formulario 1').get_all_values()

In [ ]:
type(sheets)

In [ ]:
# Convirtiendo la lista en df
import pandas as pd
df = pd.DataFrame(sheets[1:], columns=sheets[0])

## Inspección del data frame

In [ ]:
type(df)

In [ ]:
# ver el tamaño del df
df.shape

In [ ]:
# Ver las primeras filas
df.head()

In [ ]:
# Ver al estructura del df
df.info()

In [ ]:
# dtype
df['Sexo'].dtype

In [ ]:
# Número de filas
len(df)

In [ ]:
# Número de columnas
len(df.columns)

# Tema 02: Transformación de datos

## Valores perdidos

### Identificando NAs

In [ ]:
# Los NAs están como strings vacíos
import numpy as np
df.replace('', np.nan, inplace=True)

In [ ]:
df['Escribe tu edad exacta'] \
.isna() \
.value_counts()

### Tratando los NAs

In [ ]:
df['Escribe tu edad exacta'].dtype

In [ ]:
df['Escribe tu edad exacta'] = pd.to_numeric(df['Escribe tu edad exacta'], errors='coerce')

In [ ]:
df['Escribe tu edad exacta'].dtype

Reemplazo por la media

In [ ]:
edad_promedio = df['Escribe tu edad exacta'].mean()

In [ ]:
# Creando df2
df2 = df.copy()

In [ ]:
# Reemplazo por la media
df2['edad2'] = df2['Escribe tu edad exacta'].fillna(edad_promedio)

In [ ]:
df2[['Escribe tu edad exacta', 'edad2']]

Reubicando edad2

In [ ]:
# Crear una lista con los nombres de las columnas
lista_columnas = list(df2.columns)

In [ ]:
# Quitando temporalmente edad2 de la lista
lista_columnas.remove('edad2')

In [ ]:
# Obteniendo el índice (posición) de la columna "after"
lista_columnas.index('Escribe tu edad exacta')

In [ ]:
# Insertando "edad2" después de "Escribe tu edad exacta"
lista_columnas.insert(lista_columnas.index('Escribe tu edad exacta') + 1, 'edad2')

In [ ]:
df2 = df2[lista_columnas]

In [ ]:
df2.info()

Creando relocate() en py

In [ ]:
def relocate(df, columna, after):
  lista_columnas = list(df.columns)
  lista_columnas.remove(columna)
  lista_columnas.insert(lista_columnas.index(after) + 1, columna)
  return df[lista_columnas]

In [ ]:
relocate(
    df = df2,
    columna = 'edad2',
    after = 'Escribe tu edad exacta'
)

Eliminar la fila completa

In [ ]:
df2.shape

In [ ]:
df2.dropna(inplace=True)

In [ ]:
df2.shape

## Estandarización de variables

### Normalización

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# instansciando StandardScaler()
normalizador = StandardScaler()

In [ ]:
# normalizando
normalizador.fit_transform(df2[['edad2']])

In [ ]:
# creando df3
df3 = df2.copy()

In [ ]:
df3['edadZ'] = normalizador.fit_transform(df3[['edad2']])

In [ ]:
df3 = relocate(
  df = df3,
  columna = 'edadZ',
  after = 'edad2'
)

### Rango

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# instanciando MinMaxScaler()
rango = MinMaxScaler()

In [ ]:
df3['edadR'] = rango.fit_transform(df3[['edad2']])

In [ ]:
df3 = relocate(
  df = df3,
  columna = 'edadR',
  after = 'edadZ'
)

In [ ]:
df3[['edad2', 'edadZ', 'edadR']]

## Agrupaciones

### Numéricas

In [47]:
cortes = [-float('inf'), 18, 21, float('inf')]
etiquetas = ['18 o menos', '19 a 21', 'Más de 21']

In [51]:
df3['edadGR'] = pd.cut(
    x = df3['edad2'],
    bins = cortes,
    labels = etiquetas
)

In [53]:
df3 = relocate(
    df = df3,
    columna = 'edadGR',
    after = 'edadR'
)

In [55]:
df3['edadGR'].value_counts()

,count
edadGR,
Más de 21,111
19 a 21,97
18 o menos,31


### Categóricas

In [ ]:
# Revisando índices
df3.info()

In [60]:
# Valores únicos
pd.unique(df3.iloc[:,8])

array(['No lo sé', 'Totalmente verdadero', 'Un poco falso',
       'Un poco verdadero', 'Totalmente falso'], dtype=object)

In [ ]:
# Frecuencias
df3.iloc[:,8].value_counts()

In [ ]:
df3.iloc[:,8].isin(['Totalmente verdadero', 'Un poco verdadero'])

Dummitización

In [ ]:
# Función condicional normal
